# TESTING a created HDF5-FILE
This jupyter notebook is only for manual testing of a hdf5-file.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import openslide
from skimage.filters import threshold_otsu
from skimage.transform import resize
import h5py

In [ ]:
# parameters
path_to_tif = 'output_1549967798.1690197.hdf5'

### Read the file

In [ ]:
f = h5py.File(path_to_tif, mode='r')

### Check the shapes and names of the dataset

In [ ]:
list(f)

In [ ]:
f['img'].shape

In [ ]:
f['coord_relative'][:5]

### Testing the mask and tissue tile from different levels

In [ ]:
# parameters
index = 129
level_low = 0 # for the left column
level_high = 2 # for the right column

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(3,2,1)
plt.imshow(f['img'][level_low,index])
plt.subplot(3,2,2)
plt.imshow(f['img'][level_high,index])
plt.subplot(3,2,3)
plt.imshow(f['mask'][level_low,index])
plt.subplot(3,2,4)
plt.imshow(f['mask'][level_high,index])
plt.subplot(3,2,5)
l0 = np.int64(f['img'][level_low,index])
l0[:,:,1] += f['mask'][level_low,index] *100
plt.imshow(l0)
plt.subplot(3,2,6)
l1 = np.int64(f['img'][level_high,index])
l1[:,:,1] += f['mask'][level_high,index] *100
plt.imshow(l1)

# Use the information of the position to get one big tile again

In [ ]:
# parameters
img_size = 2500
layer = 0
# reduce the pixel (for example 2 reduce the size of the image by two)
reduce = 1
start_pos=1000*8

In [ ]:
num_of_samples = f['img'].shape[1]
shape_of_img = f['img'].shape[2:4]
img  = np.zeros((img_size,img_size,3),dtype=np.uint8)
mask = np.zeros((img_size,img_size),  dtype=np.bool)

for i in range(num_of_samples):
    coord_relative = f['coord_relative'][layer,i]
    ys = int(coord_relative[0] // reduce)-start_pos
    ye = int(ys + shape_of_img[0]// reduce) 
    xs = int(coord_relative[1] // reduce)-start_pos
    xe = int(xs + shape_of_img[1]// reduce) 
    if (xs < 0 or ys < 0 or xe >= img_size or ye >= img_size) == False:
        img[xs:xe,ys:ye] = f['img'][layer,i,::reduce,::reduce]
        mask[xs:xe,ys:ye] = f['mask'][layer,i,::reduce,::reduce]
print(i)
plt.figure(figsize=(15,15))
plt.imshow(img)
plt.figure(figsize=(15,15))
plt.imshow(mask)

plt.figure(figsize=(15,15))
img_mask = np.int64(img)
img_mask[:,:,1] += mask *100
plt.imshow(img_mask)